# 🇯🇵→🇰🇷 MarianMT 学習ノートブック

日本語→韓国語翻訳モデルの学習パイプライン

## 手順
1. 環境セットアップ
2. 教師データ生成 (NLLB-200)
3. MarianMT学習
4. ONNX変換・エクスポート

In [1]:
import os

## 0. GPU確認

In [2]:
!nvidia-smi

Sat Jan 24 22:39:59 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:A1:00.0 Off |                  Off |
|  0%   30C    P8             23W /  450W |       2MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 1. 環境セットアップ

In [3]:
# RunPodの永続ディレクトリを設定
base_path = "/workspace/grasp-models"
os.makedirs(base_path, exist_ok=True)
%cd {base_path}

/workspace/grasp-models


/usr/local/lib/python3.11/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
!mkdir -p /workspace/huggingface_cache
!ls /workspace/huggingface_cache

models--facebook--nllb-200-3.3B  xet


In [4]:
# キャッシュ先を容量の大きい /workspace 配下に変更
os.environ['HF_HOME'] = '/workspace/huggingface_cache'
os.environ['HUGGINGFACE_HUB_CACHE'] = '/workspace/huggingface_cache'

In [5]:
rm -rf /root/.cache/huggingface/*

In [7]:
!df -h

Filesystem                   Size  Used Avail Use% Mounted on
overlay                       20G   16G  4.8G  77% /
tmpfs                         64M     0   64M   0% /dev
mfs#us-nc-1.runpod.net:9421  1.2P  677T  525T  57% /workspace
shm                           29G     0   29G   0% /dev/shm
/dev/md127                    14T  1.2T   13T   9% /etc/hosts
/dev/nvme0n1p2               1.8T   21G  1.7T   2% /usr/bin/nvidia-smi
tmpfs                        252G     0  252G   0% /sys/fs/cgroup
tmpfs                        252G   12K  252G   1% /proc/driver/nvidia
tmpfs                        252G  4.0K  252G   1% /etc/nvidia/nvidia-application-profiles-rc.d
tmpfs                         51G   33M   51G   1% /run/nvidia-persistenced/socket
tmpfs                        252G     0  252G   0% /proc/asound
tmpfs                        252G     0  252G   0% /proc/acpi
tmpfs                        252G     0  252G   0% /proc/scsi
tmpfs                        252G     0  252G   0% /sys/firmware
tmpfs

In [8]:
!pwd

/workspace/grasp-models


In [6]:
# リポジトリのクローン
# すでにフォルダが存在する場合は pull して最新にします
repo_url = "https://github.com/nakaikento/grasp-models.git"
repo_dir = "grasp-models"

if not os.path.exists(repo_dir):
    print(f"Cloning repository from {repo_url}...")
    !git clone {repo_url}
    %cd {repo_dir}
else:
    print("Repository already exists. Pulling latest changes...")
    %cd {repo_dir}
    !git pull

Repository already exists. Pulling latest changes...
/workspace/grasp-models/grasp-models
Already up to date.


In [34]:
# 依存関係インストール
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


## 2. データ確認

In [12]:
# データファイル確認
!ls -la data/splits/
!wc -l data/splits/*.ja data/splits/*.ko

total 84484
drwxrwxrwx 2 root root  2007865 Jan 24 08:24 .
drwxrwxrwx 7 root root  2024298 Jan 24 08:27 ..
-rw-rw-rw- 1 root root   199447 Jan 24 08:24 test.ja
-rw-rw-rw- 1 root root   200408 Jan 24 08:24 test.ko
-rw-rw-rw- 1 root root 40801930 Jan 24 08:24 train.ja
-rw-rw-rw- 1 root root 40879416 Jan 24 08:24 train.ko
-rw-rw-rw- 1 root root   198145 Jan 24 08:24 val.ja
-rw-rw-rw- 1 root root   197178 Jan 24 08:24 val.ko
    4999 data/splits/test.ja
 1025748 data/splits/train.ja
    4999 data/splits/val.ja
    4999 data/splits/test.ko
 1025748 data/splits/train.ko
    4999 data/splits/val.ko
 2071492 total


In [13]:
# トークナイザー確認
!ls -la data/tokenized/

total 5338
drwxrwxrwx 2 root root 2000137 Jan 24 08:24 .
drwxrwxrwx 7 root root 2024298 Jan 24 08:27 ..
-rw-rw-rw- 1 root root  825950 Jan 24 08:24 spm.model
-rw-rw-rw- 1 root root  614203 Jan 24 08:24 spm.vocab


## 3. 教師データ生成（Knowledge Distillation用）

NLLB-200を使って高品質な韓国語翻訳を生成

In [46]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 1.11 KiB | 56.00 KiB/s, done.
From https://github.com/nakaikento/grasp-models
   25e03b3..a5eab12  main       -> origin/main
Updating 25e03b3..a5eab12
Fast-forward
 training/generate_teacher_data.py | 43 +++++++++++++++++++++++----------------
 1 file changed, 26 insertions(+), 17 deletions(-)


In [49]:
rm data/teacher/train.ko

In [50]:
!ls data/teacher/train.ko

ls: cannot access 'data/teacher/train.ko': No such file or directory


In [53]:
# 教師データ生成（約8時間）
!python training/generate_teacher_data.py --batch_size 64

🚀 超高品質モード(16-bit + Beam3 + Cleaning)でロード中...
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████████████| 3/3 [00:32<00:00, 10.77s/it]
🔄 268288行目から再開します...
 29%|██████████▌                         | 4219/14380 [00:42<2:51:34,  1.01s/it]
--- [進捗チェック: 270016行目] ---
原: まず、ソフトをアップグレードして差し上げよう、 そして...
洗: まず、ソフトをアップグレードして差し上げよう、 そして...
韓: FAILED_TRANSLATION_CLEANED
----------------------------------------
 30%|██████████▊                         | 4297/14380 [01:51<2:21:08,  1.19it/s]
--- [進捗チェック: 275008行目] ---
原: かけ直す
洗: かけ直す
韓: FAILED_TRANSLATION_CLEANED
----------------------------------------
 30%|██████████▉                         | 4375/14380 [02:58<2:09:04,  1.29it/s]
--- [進捗チェック: 280000行目] ---
原: フランキーが電話を？
洗: フランキーが電話を？
韓: FAILED_TRANSLATION_CLEANED
----------------------------------------
 31%|███████████▏                        | 4454/14380 [04:04<2:15:09,  1.22it/s]
--- [進捗チェック: 285056行目] ---
原: よく気がつく性質でね 細かいことは気にせず 話を聞くんだ
洗: よく気がつく性質でね 

In [54]:
def clean_and_sync():
    ja_in = "data/raw/OpenSubtitles.ja-ko.ja"
    ko_in = "data/teacher/train.ko"
    
    ja_out = "data/clean/train.ja"
    ko_out = "data/clean/train.ko"
    
    os.makedirs("data/clean", exist_ok=True)

    with open(ja_in, 'r', encoding='utf-8') as f_ja, \
         open(ko_in, 'r', encoding='utf-8') as f_ko, \
         open(ja_out, 'w', encoding='utf-8') as o_ja, \
         open(ko_out, 'w', encoding='utf-8') as o_ko:
        
        for ja_line, ko_line in zip(f_ja, f_ko):
            ja_line = ja_line.strip()
            ko_line = ko_line.strip()
            
            # 失敗行、空行、またはエラー文字列が含まれる場合はスキップ
            if "FAILED_TRANSLATION_CLEANED" in ko_line or not ko_line:
                continue
            
            o_ja.write(ja_line + "\n")
            o_ko.write(ko_line + "\n")

    print(f"✨ クリーニング完了！")
    print(f"📁 保存先: data/clean/train.ja, train.ko")

if __name__ == "__main__":
    clean_and_sync()

✨ クリーニング完了！
📁 保存先: data/clean/train.ja, train.ko


In [30]:
# !rm data/teacher/train.ko
# !wc -l data/teacher/train.ko

In [56]:
!wc -l data/clean/train.ko
!wc -l data/clean/train.ja

605964 data/clean/train.ko
605964 data/clean/train.ja


In [57]:
import random

def verify_parallel_data(ja_path, ko_path, num_samples=10):
    with open(ja_path, 'r', encoding='utf-8') as f_ja:
        ja_lines = [line.strip() for line in f_ja]
    with open(ko_path, 'r', encoding='utf-8') as f_ko:
        ko_lines = [line.strip() for line in f_ko]

    total = len(ja_lines)
    print("="*50)
    print(f"📊 データセット統計")
    print(f"  総ペア数: {total:,} 行")
    print("="*50)

    # ランダムにサンプルを抽出
    indices = random.sample(range(total), num_samples)
    
    print(f"\n🔍 ランダムサンプルチェック ({num_samples}件)")
    print("-" * 50)
    for idx in sorted(indices):
        ja = ja_lines[idx]
        ko = ko_lines[idx]
        
        # 簡易的な品質指標
        # 日本語と韓国語の長さの比率（極端に違う場合は注意）
        ratio = len(ko) / len(ja) if len(ja) > 0 else 0
        status = "✅" if 0.3 < ratio < 3.0 else "⚠️"

        print(f"[{idx+1}行目] {status}")
        print(f"  日: {ja}")
        print(f"  韓: {ko}")
        print(f"  (長さ比: {ratio:.2f})")
        print("-" * 30)

    # 最も長い文のチェック（バグ発見用）
    print("\n超長文チェック (異常な繰り返しがないか)")
    max_idx = max(range(total), key=lambda i: len(ja_lines[i]))
    print(f"[{max_idx+1}行目]")
    print(f"  日: {ja_lines[max_idx][:100]}...")
    print(f"  韓: {ko_lines[max_idx][:100]}...")

if __name__ == "__main__":
    verify_parallel_data("data/clean/train.ja", "data/clean/train.ko")

📊 データセット統計
  総ペア数: 605,964 行

🔍 ランダムサンプルチェック (10件)
--------------------------------------------------
[10353行目] ✅
  日: あんたが私をこんなにした
  韓: 넌 나를 이렇게 만들었지
  (長さ比: 1.08)
------------------------------
[18043行目] ✅
  日: それで何がわかったと思う？
  韓: 그래서 뭘 알았다고 생각하나?
  (長さ比: 1.23)
------------------------------
[228716行目] ✅
  日: コヨーテでいいのか？ - そうだ　ハンク
  韓: 코요테는 어때?
  (長さ比: 0.40)
------------------------------
[249598行目] ✅
  日: エイリアンは居ないわ
  韓: 에일리안은 여기 없어
  (長さ比: 1.10)
------------------------------
[260208行目] ✅
  日: - 誰かが支払わないとな
  韓: 누가 돈을 안 내면
  (長さ比: 0.83)
------------------------------
[316574行目] ✅
  日: やるぞ
  韓: 어서!
  (長さ比: 1.00)
------------------------------
[392388行目] ✅
  日: ええ ここで何をやっているんだ？ 聞かないと
  韓: 여기서 뭐하는 거야?
  (長さ比: 0.50)
------------------------------
[485977行目] ✅
  日: そんな事はない
  韓: 그런건 없지
  (長さ比: 0.86)
------------------------------
[530050行目] ✅
  日: やあ　どうも レックスです...
  韓: 안녕하세요, 렉스...
  (長さ比: 0.75)
------------------------------
[589965行目] ✅
  日: 彼は2番目のモジュールを作ったので 惑星全体を制御できます
  韓: 그는 두 번째 모듈을 

In [62]:
# ノートブックのセルで実行
import shutil

# NLLBモデルのキャッシュディレクトリ（RunPodの標準的なパス）
cache_dir = "/workspace/huggingface_cache/models--facebook--nllb-200-3.3b"

if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print(f"✅ {cache_dir} を削除しました。空き容量が大幅に増えたはずです。")
else:
    print("❌ 指定のパスにキャッシュが見つかりませんでした。")

✅ /workspace/huggingface_cache/xet を削除しました。空き容量が大幅に増えたはずです。


In [64]:
!df -h

Filesystem                   Size  Used Avail Use% Mounted on
overlay                       20G   16G  4.7G  77% /
tmpfs                         64M     0   64M   0% /dev
mfs#us-nc-1.runpod.net:9421  1.2P  684T  518T  57% /workspace
shm                           29G     0   29G   0% /dev/shm
/dev/md127                    14T  1.2T   13T   9% /etc/hosts
/dev/nvme0n1p2               1.8T   20G  1.7T   2% /usr/bin/nvidia-smi
tmpfs                        252G     0  252G   0% /sys/fs/cgroup
tmpfs                        252G   12K  252G   1% /proc/driver/nvidia
tmpfs                        252G  4.0K  252G   1% /etc/nvidia/nvidia-application-profiles-rc.d
tmpfs                         51G   33M   51G   1% /run/nvidia-persistenced/socket
tmpfs                        252G     0  252G   0% /proc/asound
tmpfs                        252G     0  252G   0% /proc/acpi
tmpfs                        252G     0  252G   0% /proc/scsi
tmpfs                        252G     0  252G   0% /sys/firmware
tmpfs

In [13]:
ls -la /workspace/huggingface_cache/

total 9836
drwxrwxrwx 5 root root 3001639 Jan 24 08:35 ./
drwxrwxrwx 5 root root 3001672 Jan 24 22:40 ../
drwxrwxrwx 3 root root       1 Jan 24 08:35 .locks/
drwxrwxrwx 6 root root 3001639 Jan 24 08:46 models--facebook--nllb-200-3.3B/
drwxrwxrwx 4 root root 1065397 Jan 24 08:35 xet/


## 4. MarianMT学習

In [18]:
!pwd

/workspace/grasp-models/grasp-models


In [20]:
!ls /workspace/grasp-models/grasp-models/

README.md  docs    gitignore	     run.sh   setup.sh
data	   export  requirements.txt  scripts  training


In [23]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 24.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split

# データの読み込み
with open('data/clean/train.ja', 'r', encoding='utf-8') as f:
    ja_data = f.readlines()
with open('data/clean/train.ko', 'r', encoding='utf-8') as f:
    ko_data = f.readlines()

# 行数が一致しているか最終確認
if len(ja_data) == len(ko_data):
    print(f"✅ 行数一致確認: {len(ja_data)} 行")
    
    # データを分割 (学習用 98%, 検証用 1%, テスト用 1%)
    # 100万行あるので、各1%（約1万行）あれば評価には十分です
    train_ja, temp_ja, train_ko, temp_ko = train_test_split(ja_data, ko_data, test_size=0.02, random_state=42)
    val_ja, test_ja, val_ko, test_ko = train_test_split(temp_ja, temp_ko, test_size=0.5, random_state=42)

    # 保存用ディレクトリ作成
    os.makedirs('data/final', exist_ok=True)

    # 各ファイルを保存
    def save_list(path, data):
        with open(path, 'w', encoding='utf-8') as f:
            f.writelines(data)

    save_list('data/final/train.ja', train_ja)
    save_list('data/final/train.ko', train_ko)
    save_list('data/final/val.ja', val_ja)
    save_list('data/final/val.ko', val_ko)
    save_list('data/final/test.ja', test_ja)
    save_list('data/final/test.ko', test_ko)
    
    print("✅ データの分割と保存が完了しました！")
else:
    print(f"❌ 行数が一致しません: JA={len(ja_data)}, KO={len(ko_data)}")

✅ 行数一致確認: 605964 行
✅ データの分割と保存が完了しました！


In [67]:
def check_alignment(ja_path, ko_path, num_samples=5):
    with open(ja_path, 'r', encoding='utf-8') as f_ja, \
         open(ko_path, 'r', encoding='utf-8') as f_ko:
        # メモリ節約のためreadlinesではなく、最小行数まで読み込み
        ja_lines = f_ja.readlines()
        ko_lines = f_ko.readlines()

    total_ja = len(ja_lines)
    total_ko = len(ko_lines)
    
    print(f"--- 統計 ---")
    print(f"日本語行数: {total_ja}")
    print(f"韓国語行数: {total_ko}")
    print(f"差分: {abs(total_ja - total_ko)}\n")

    def print_samples(indices, label):
        print(f"--- {label} のサンプル ---")
        for i in indices:
            if i < len(ja_lines) and i < len(ko_lines):
                print(f"[{i+1}行目]")
                print(f"日: {ja_lines[i].strip()}")
                print(f"韓: {ko_lines[i].strip()}")
                print("-" * 30)

    # 1. 先頭 (1〜5行目)
    print_samples(range(0, num_samples), "先頭")
    
    # 2. 中間 (50万行目あたり)
    middle_idx = 500000
    print_samples(range(middle_idx, middle_idx + num_samples), "中間")
    
    # 3. 末尾（韓国語データの最後の方）
    end_idx = total_ko - num_samples
    print_samples(range(end_idx, end_idx + num_samples), "末尾（同期可能範囲）")

# 正しいパスで実行
check_alignment('data/clean/train.ja', 'data/clean/train.ko')

--- 統計 ---
日本語行数: 605964
韓国語行数: 605964
差分: 0

--- 先頭 のサンプル ---
[1行目]
日: 昔の人は言っていた
韓: 옛날에 사람들이 말했지
------------------------------
[2行目]
日: この物語のヒロイン
韓: 이 이야기의 히로인
------------------------------
[3行目]
日: "１ドル"
韓: "1달러"
------------------------------
[4行目]
日: "３ドル"
韓: 3달러
------------------------------
[5行目]
日: １ドル落ちてなかった？
韓: 1달러 떨어졌어요?
------------------------------
--- 中間 のサンプル ---
[500001行目]
日: 我々の行動も中止
韓: 우리의 활동도 중지
------------------------------
[500002行目]
日: そのデバイスが動いたら 大勢が死にます
韓: 그 장치가 움직이면, 많은 사람들이 죽는다.
------------------------------
[500003行目]
日: 何を？
韓: 뭐야?
------------------------------
[500004行目]
日: あなたが "私はテイラーだ" と言ってから...
韓: "나는 테일러다"라고 말하는데...
------------------------------
[500005行目]
日: それが意味している事を 考えてたけど...
韓: 그게 무슨 뜻인지 생각해봤는데...
------------------------------
--- 末尾（同期可能範囲） のサンプル ---
[605960行目]
日: 開かれた...
韓: 열렸어
------------------------------
[605961行目]
日: （森口）これはねえ
韓: 이건 안돼
------------------------------
[605962行目]
日: ありがとうございます
韓: 감사합니다.
------------------------------
[

In [86]:
# テストデータ分割

# 読み込み
with open('data/clean/train.ja', 'r') as f:
    ja_lines = f.readlines()
with open('data/clean/train.ko', 'r') as f:
    ko_lines = f.readlines()

total = len(ja_lines)
train_end = total * 95 // 100
val_end = total * 975 // 1000

print(f"Total: {total}")
print(f"Train: 0 - {train_end} ({train_end} lines)")
print(f"Val: {train_end} - {val_end} ({val_end - train_end} lines)")
print(f"Test: {val_end} - {total} ({total - val_end} lines)")

# 分割して保存
with open('data/clean/train.ja', 'w') as f:
    f.writelines(ja_lines[:train_end])
with open('data/clean/train.ko', 'w') as f:
    f.writelines(ko_lines[:train_end])

with open('data/clean/val.ja', 'w') as f:
    f.writelines(ja_lines[train_end:val_end])
with open('data/clean/val.ko', 'w') as f:
    f.writelines(ko_lines[train_end:val_end])

with open('data/clean/test.ja', 'w') as f:
    f.writelines(ja_lines[val_end:])
with open('data/clean/test.ko', 'w') as f:
    f.writelines(ko_lines[val_end:])

print("\n分割完了!")

Total: 575665
Train: 0 - 546881 (546881 lines)
Val: 546881 - 561273 (14392 lines)
Test: 561273 - 575665 (14392 lines)

分割完了!


In [96]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 1.62 KiB | 51.00 KiB/s, done.
From https://github.com/nakaikento/grasp-models
   66ed2d8..40d7525  main       -> origin/main
Updating 66ed2d8..40d7525
Fast-forward
 training/train.py | 96 +++++++++++++++++++++++++++++++++++++++++++++++++------
 1 file changed, 87 insertions(+), 9 deletions(-)


In [77]:
from huggingface_hub import notebook_login

notebook_login()

In [88]:
from huggingface_hub import whoami

try:
    user_info = whoami()
    print(f"✅ ログイン成功: {user_info['name']} として認証されています")
except Exception as e:
    print(f"❌ ログインできていないか、エラーが発生しました: {e}")

✅ ログイン成功: ken4869 として認証されています


In [89]:
# wandbログイン（オプション）
import wandb
wandb.login()

True

In [90]:
!ls data/clean

test.ja  test.ko  train.ja  train.ko  val.ja  val.ko


In [97]:
# 学習実行（教師データ使用）
!python training/train.py \
    --data-dir data/clean \
    --tokenizer data/tokenized/spm.model \
    --output-dir /workspace/models/ja-ko \
    --epochs 10 \
    --batch-size 128 \
    --learning-rate 3e-4 \
    --num-workers 12

MarianMT 学習
デバイス: cuda
GPU: NVIDIA GeForce RTX 4090

トークナイザーをロード: data/tokenized/spm.model
Vocab size: 32000

データをロード...
ターゲット: OPUS韓国語
Train: 546,881
Val:   14,392
Test:  14,392

前処理中...
Tokenizing: 100%|███████████████| 14392/14392 [00:00<00:00, 97061.55 examples/s]

モデルを作成...
モデルパラメータ数: 61,046,784 (61.0M)

学習開始...
  エポック: 10
  バッチサイズ: 128 x 2 = 256
  学習率: 0.0003
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.
wandb: Currently logged in as: okamoto2okamoto (okamoto2okamoto-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ Waiting for wandb.init()...
wandb: Tracking run with wandb version 0.24.0
wandb: Run data is saved locally in /workspace/grasp-models/grasp-models/wandb/run-20260125_082514-zsh840l3
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run deft-monkey-8
wandb: ⭐️ View project at https://wandb.ai/okamoto2okamoto-personal/huggingface
wandb: 🚀 View

In [ ]:
# または：OPUSデータで学習（教師データなし、比較用）
# !python training/train.py \
#     --use-opus-target \
#     --output-dir /content/drive/MyDrive/grasp-models/models/ja-ko-opus \
#     --epochs 10 \
#     --batch-size 32

## 5. モデル評価

In [99]:
import torch
from pathlib import Path
import sys
sys.path.append('/workspace/grasp-models/grasp-models')

from transformers import MarianMTModel
from training.train import SPMTokenizer

# モデルとトークナイザーをロード
model_path = "/workspace/models/ja-ko"
model = MarianMTModel.from_pretrained(model_path)
tokenizer = SPMTokenizer(f"{model_path}/spm.model")

# GPU使用
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128, num_beams=4)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# テスト翻訳
test_sentences = [
    # ドラマ定番
    "私を信じて。",
    "嘘をつかないで。",
    "もう一度チャンスをください。",
    "君のことが好きだ。",
    "行かないで！",
    
    # 感情的なシーン
    "どうしてそんなことを言うの？",
    "全部私のせいだ。",
    "君がいないと生きていけない。",
    "許してくれ。",
    "二度と会いたくない。",
    
    # アクション系
    "逃げろ！",
    "後ろに誰かいる！",
    "時間がない。急げ！",
    "俺に任せろ。",
    "絶対に諦めない。",
    
    # 日常ドラマ
    "今日は残業？",
    "先輩、飲みに行きませんか？",
    "彼氏と別れたの。",
    "結婚してください。",
    "お母さん、ごめんなさい。",
]

for ja in test_sentences:
    ko = translate(ja)
    print(f"🇯🇵 {ja}")
    print(f"🇰🇷 {ko}")
    print()

🇯🇵 私を信じて。
🇰🇷 나 믿어요

🇯🇵 嘘をつかないで。
🇰🇷 거짓말 하지 마

🇯🇵 もう一度チャンスをください。
🇰🇷 다시 한번 기회를 주세요.

🇯🇵 君のことが好きだ。
🇰🇷 너에 대해 좋아해

🇯🇵 行かないで！
🇰🇷 가자!

🇯🇵 どうしてそんなことを言うの？
🇰🇷 왜 그런 말 하는 거야?

🇯🇵 全部私のせいだ。
🇰🇷 다 내 탓이야

🇯🇵 君がいないと生きていけない。
🇰🇷 네가 없으면 살아갈 수 없어.

🇯🇵 許してくれ。
🇰🇷 용서해줄게

🇯🇵 二度と会いたくない。
🇰🇷 다시는 만나고 싶지 않아.

🇯🇵 逃げろ！
🇰🇷 도망쳐!

🇯🇵 後ろに誰かいる！
🇰🇷 뒤에 누가 있어!

🇯🇵 時間がない。急げ！
🇰🇷 시간 없어!

🇯🇵 俺に任せろ。
🇰🇷 나한테 맡겨라

🇯🇵 絶対に諦めない。
🇰🇷 절대 포기하지 않을거야.

🇯🇵 今日は残業？
🇰🇷 오늘 남았어요?

🇯🇵 先輩、飲みに行きませんか？
🇰🇷 먼저 마실 수 있나요?

🇯🇵 彼氏と別れたの。
🇰🇷 남자친구와 헤어졌어요

🇯🇵 結婚してください。
🇰🇷 결혼하세요.

🇯🇵 お母さん、ごめんなさい。
🇰🇷 어머님, 죄송합니다.



## 6. ONNX変換（Android用）

In [101]:
# 依存関係インストール
!pip install -q optimum[onnxruntime]


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [102]:
# ONNX変換
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from pathlib import Path
import shutil

model_path = "/workspace/models/ja-ko"
output_path = "/workspace/models/ja-ko-onnx"

# 変換
print("ONNX変換中...")
ort_model = ORTModelForSeq2SeqLM.from_pretrained(
    model_path,
    export=True
)

# 保存
ort_model.save_pretrained(output_path)

# トークナイザーもコピー
shutil.copy(f"{model_path}/spm.model", f"{output_path}/spm.model")

print(f"✅ 完了: {output_path}")

# サイズ確認
!ls -lh {output_path}/
!du -sh {output_path}/

`torch_dtype` is deprecated! Use `dtype` instead!


ONNX変換中...


/usr/local/lib/python3.11/dist-packages/transformers/modeling_attn_mask_utils.py:196: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  inverted_mask = torch.tensor(1.0, dtype=dtype) - expanded_mask
/usr/local/lib/python3.11/dist-packages/transformers/integrations/sdpa_attention.py:81: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  is_causal = query.shape[2] > 1 and attention_mask is None and getattr(module, "is_causal", True)
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/modeling_marian.py:642: Tra

✅ 完了: /workspace/models/ja-ko-onnx
total 570M
-rw-rw-rw- 1 root root  876 Jan 25 09:31 config.json
-rw-rw-rw- 1 root root 223M Jan 25 09:31 decoder_model.onnx
-rw-rw-rw- 1 root root 211M Jan 25 09:31 decoder_with_past_model.onnx
-rw-rw-rw- 1 root root 136M Jan 25 09:31 encoder_model.onnx
-rw-rw-rw- 1 root root  140 Jan 25 09:31 generation_config.json
-rw-rw-rw- 1 root root 807K Jan 25 09:31 spm.model
572M	/workspace/models/ja-ko-onnx/


In [103]:
# 変換後のテスト
from optimum.onnxruntime import ORTModelForSeq2SeqLM
import sys
sys.path.append('/workspace/grasp-models/grasp-models')
from training.train import SPMTokenizer

# ONNXモデルをロード
ort_model = ORTModelForSeq2SeqLM.from_pretrained(output_path)
tokenizer = SPMTokenizer(f"{output_path}/spm.model")

def translate_onnx(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    outputs = ort_model.generate(**inputs, max_length=128, num_beams=4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# テスト
test = [
    "こんにちは",
    "逃げろ！",
    "君のことが好きだ。",
]

for ja in test:
    ko = translate_onnx(ja)
    print(f"🇯🇵 {ja} → 🇰🇷 {ko}")

Could not find any ONNX files with standard file name decoder_model_merged.onnx, files found: [PosixPath('decoder_with_past_model.onnx'), PosixPath('decoder_model.onnx'), PosixPath('encoder_model.onnx')]. Please make sure to pass a `file_name` and/or `subfolder` argument to `from_pretrained` when loading an ONNX file with non-standard file names.


🇯🇵 こんにちは → 🇰🇷 안녕하세요
🇯🇵 逃げろ！ → 🇰🇷 도망쳐!
🇯🇵 君のことが好きだ。 → 🇰🇷 너에 대해 좋아해


In [104]:
# 量子化して軽量化
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig
import shutil
from pathlib import Path

output_path = "/workspace/models/ja-ko-onnx"
quantized_path = "/workspace/models/ja-ko-onnx-int8"

Path(quantized_path).mkdir(exist_ok=True)

# 量子化設定
qconfig = AutoQuantizationConfig.avx512_vnni(is_static=False)

# 全ONNXファイルを量子化
onnx_files = [
    "encoder_model.onnx",
    "decoder_model.onnx", 
    "decoder_with_past_model.onnx"
]

for onnx_file in onnx_files:
    print(f"量子化中: {onnx_file}")
    quantizer = ORTQuantizer.from_pretrained(output_path, file_name=onnx_file)
    quantizer.quantize(save_dir=quantized_path, quantization_config=qconfig)

# 設定ファイルとトークナイザーをコピー
shutil.copy(f"{output_path}/config.json", quantized_path)
shutil.copy(f"{output_path}/generation_config.json", quantized_path)
shutil.copy(f"{output_path}/spm.model", quantized_path)

print("\n✅ 量子化完了!")

# サイズ確認
!ls -lh {quantized_path}/
!du -sh {quantized_path}/

量子化中: encoder_model.onnx
量子化中: decoder_model.onnx
量子化中: decoder_with_past_model.onnx

✅ 量子化完了!
total 146M
-rw-rw-rw- 1 root root  876 Jan 25 09:34 config.json
-rw-rw-rw- 1 root root  57M Jan 25 09:34 decoder_model_quantized.onnx
-rw-rw-rw- 1 root root  54M Jan 25 09:34 decoder_with_past_model_quantized.onnx
-rw-rw-rw- 1 root root  35M Jan 25 09:34 encoder_model_quantized.onnx
-rw-rw-rw- 1 root root  140 Jan 25 09:34 generation_config.json
-rw-rw-rw- 1 root root  762 Jan 25 09:34 ort_config.json
-rw-rw-rw- 1 root root 807K Jan 25 09:34 spm.model
148M	/workspace/models/ja-ko-onnx-int8/


In [106]:
from optimum.onnxruntime import ORTModelForSeq2SeqLM
import sys
sys.path.append('/workspace/grasp-models/grasp-models')
from training.train import SPMTokenizer

# 量子化モデルをロード
quantized_path = "/workspace/models/ja-ko-onnx-int8"
ort_model = ORTModelForSeq2SeqLM.from_pretrained(quantized_path)
tokenizer = SPMTokenizer(f"{quantized_path}/spm.model")

def translate_q(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    outputs = ort_model.generate(**inputs, max_length=128, num_beams=4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 量子化前後で品質比較
test = [
    # ドラマ定番
    "私を信じて。",
    "嘘をつかないで。",
    "もう一度チャンスをください。",
    "君のことが好きだ。",
    "行かないで！",
    
    # 感情的なシーン
    "どうしてそんなことを言うの？",
    "全部私のせいだ。",
    "君がいないと生きていけない。",
    "許してくれ。",
    "二度と会いたくない。",
    
    # アクション系
    "逃げろ！",
    "後ろに誰かいる！",
    "時間がない。急げ！",
    "俺に任せろ。",
    "絶対に諦めない。",
    
    # 日常ドラマ
    "今日は残業？",
    "先輩、飲みに行きませんか？",
    "彼氏と別れたの。",
    "結婚してください。",
    "お母さん、ごめんなさい。",
]

print("【量子化モデル テスト】\n")
for ja in test:
    ko = translate_q(ja)
    print(f"🇯🇵 {ja}")
    print(f"🇰🇷 {ko}")
    print()

Could not find any ONNX files with standard file name encoder_model.onnx, files found: [PosixPath('decoder_with_past_model_quantized.onnx'), PosixPath('decoder_model_quantized.onnx'), PosixPath('encoder_model_quantized.onnx')]. Please make sure to pass a `file_name` and/or `subfolder` argument to `from_pretrained` when loading an ONNX file with non-standard file names.
Could not find any ONNX files with standard file name decoder_model_merged.onnx, files found: [PosixPath('decoder_with_past_model_quantized.onnx'), PosixPath('decoder_model_quantized.onnx'), PosixPath('encoder_model_quantized.onnx')]. Please make sure to pass a `file_name` and/or `subfolder` argument to `from_pretrained` when loading an ONNX file with non-standard file names.
Could not find any ONNX files with standard file name decoder_model.onnx, files found: [PosixPath('decoder_with_past_model_quantized.onnx'), PosixPath('decoder_model_quantized.onnx'), PosixPath('encoder_model_quantized.onnx')]. Please make sure to p

【量子化モデル テスト】

🇯🇵 私を信じて。
🇰🇷 나 믿어요

🇯🇵 嘘をつかないで。
🇰🇷 거짓말 하지 마

🇯🇵 もう一度チャンスをください。
🇰🇷 다시 한번 기회를 주세요.

🇯🇵 君のことが好きだ。
🇰🇷 너에 대해 좋아해

🇯🇵 行かないで！
🇰🇷 가자!

🇯🇵 どうしてそんなことを言うの？
🇰🇷 왜 그런 말 하는 거야?

🇯🇵 全部私のせいだ。
🇰🇷 다 내 탓이야

🇯🇵 君がいないと生きていけない。
🇰🇷 네가 없으면 살 수 없어.

🇯🇵 許してくれ。
🇰🇷 용서해줄게

🇯🇵 二度と会いたくない。
🇰🇷 다시는 만나고 싶지 않아.

🇯🇵 逃げろ！
🇰🇷 도망쳐!

🇯🇵 後ろに誰かいる！
🇰🇷 뒤에 누가 있어!

🇯🇵 時間がない。急げ！
🇰🇷 시간 없어!

🇯🇵 俺に任せろ。
🇰🇷 나한테 맡겨라

🇯🇵 絶対に諦めない。
🇰🇷 절대 포기하지 않을거야.

🇯🇵 今日は残業？
🇰🇷 오늘 남았어요?

🇯🇵 先輩、飲みに行きませんか？
🇰🇷 전신 안 마셔도 될까요?

🇯🇵 彼氏と別れたの。
🇰🇷 남자친구와 헤어졌어요

🇯🇵 結婚してください。
🇰🇷 결혼하세요.

🇯🇵 お母さん、ごめんなさい。
🇰🇷 어머님, 죄송합니다



## 7. ダウンロード

学習済みモデルは `workspace/models/` に保存されています。

必須ファイル:
- encoder_model_quantized.onnx (35MB)
- decoder_model_quantized.onnx (57MB)
- spm.model (807KB)

高速推論用（推奨）:

- decoder_with_past_model_quantized.onnx (54MB)

設定ファイル:

- config.json
- generation_config.json